# LDA

In [15]:
import pandas as pd

ds = pd.read_parquet('./results/ds.parquet')
elapse_time = pd.read_csv('./results/elapse_time.csv')

In [16]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=2) # FIXME: change n to 20

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

In [17]:
from tqdm import tqdm

def save_models_lda():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'B' in v: 
            r_time.append(0)
            continue
        model = lda(ds[v])
        model['model'].save(f'./results/models_lda/{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [18]:
save_models_lda()

Training dLWN:   0%|          | 0/4 [00:00<?, ?it/s]

Training DSFN: 100%|██████████| 4/4 [02:12<00:00, 33.23s/it]


In [19]:
elapse_time

,variant,nlp,preprocessing,lda_training
0,dLWN,224.184062,7.460976,31.896072
1,DLWN,224.184062,8.284982,32.851871
2,dSFN,224.184062,1.699700,35.446574
3,DSFN,224.184062,2.410262,31.569406


# BERTopic

In [20]:
from time import time

from bertopic import BERTopic
import pandas as pd

embedding_model = "paraphrase-multilingual-MiniLM-L12-v2"

def save_models_bertopic():
    path_prefix = './results/models_bertopic/'
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        if 'T' in v: 
            r_time.append(0)
            continue
        tds.set_description(f'Training {v}')
        t_start = time()
        docs = ds[v].dropna()
        docs = [' '.join(doc) for doc in docs] if 'B' not in v else docs
        bertopic = BERTopic(language='multilingual', embedding_model=embedding_model)
        model = bertopic.fit(docs)
        t_end = time()
        model.save(
            f"{path_prefix}{v}",
            serialization="safetensors",
            save_embedding_model=embedding_model,
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [21]:
save_models_bertopic()

Training DSFN: 100%|██████████| 4/4 [09:17<00:00, 139.29s/it]


In [22]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training
0,dLWN,224.184062,7.460976,31.896072,259.104044
1,DLWN,224.184062,8.284982,32.851871,59.923101
2,dSFN,224.184062,1.699700,35.446574,178.024530
3,DSFN,224.184062,2.410262,31.569406,59.789309


In [23]:
elapse_time.to_csv('./results/elapse_time.csv', index=False)